# Covid 19 Dashboard

Import required python packages

In [1]:
import pandas as pd
from datetime import datetime

# Graph packages
import plotly.express as px
import plotly.graph_objects as go

# Dash web app packages
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
#import dash_bootstrap_components as dbc


Load CSVs into a dataframe for easier manipulation

In [2]:
country_continent = pd.read_csv('country-and-continent.csv')
country_continent.head()

,Continent_Name,CountryCode
0,Asia,AFG
1,Europe,ALB
2,Antarctica,ATA
3,Africa,DZA
4,Oceania,ASM


As this dataframe will be involved in a merge it is good to understand the shape before and after the tables are merged to check whether the merge was conducted correctly.

In [3]:
country_continent.shape

(262, 2)

Determine whether there are any null values in any of the columns and the data type of each column

In [4]:
country_continent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Continent_Name  262 non-null    object
 1   CountryCode     258 non-null    object
dtypes: object(2)
memory usage: 4.2+ KB


Locate the country codes with missing values

In [5]:
country_continent.loc[country_continent.CountryCode.isnull()]

,Continent_Name,CountryCode
258,Oceania,NaN
259,Asia,NaN
260,Asia,NaN
261,Asia,NaN


Check the shape of dataframe before dropping any rows

In [6]:
country_continent.shape

(262, 2)

As these missing values provide no information they can be removed from the dataframe

In [7]:
country_continent = country_continent.dropna()

The shape of the dataframe should now be four less than the original shape before the duplicates were dropped. 

In [8]:
country_continent.shape

(258, 2)

Load the OxCGRT_summary20200520 csv into a dataframe

In [9]:
OxCGRT_df = pd.read_csv('OxCGRT_summary20200520.csv')
OxCGRT_df.head()

,CountryName,CountryCode,Date,School closing,Stay at home requirements,ConfirmedCases,ConfirmedDeaths,StringencyIndex
0,Aruba,ABW,20200301,0,0,NaN,NaN,0.0
1,Aruba,ABW,20200302,0,0,NaN,NaN,0.0
2,Aruba,ABW,20200303,0,0,NaN,NaN,0.0
3,Aruba,ABW,20200304,0,0,NaN,NaN,0.0
4,Aruba,ABW,20200305,0,0,NaN,NaN,0.0


As this dataframe will be involved in a merge it is good to understand the shape before and after the tables are merged

In [10]:
OxCGRT_df.shape

(14013, 8)

Determine whether there are any null values in any of the columns and the data type of each column

In [11]:
OxCGRT_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14013 entries, 0 to 14012
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CountryName                14013 non-null  object 
 1   CountryCode                14013 non-null  object 
 2   Date                       14013 non-null  int64  
 3   School closing             14013 non-null  int64  
 4   Stay at home requirements  14013 non-null  int64  
 5   ConfirmedCases             11910 non-null  float64
 6   ConfirmedDeaths            11910 non-null  float64
 7   StringencyIndex            14013 non-null  float64
dtypes: float64(3), int64(3), object(2)
memory usage: 875.9+ KB


There are missing values in the ConfirmedCases and ConfirmedDeaths columns. These will be handled later.

Create a new dataframe by merging the two dataframes. 

In [12]:
OxCGRT_cont_df = OxCGRT_df.merge(country_continent, left_on = ['CountryCode'], right_on = ['CountryCode'], how = 'left')

In [13]:
OxCGRT_cont_df.head()

,CountryName,CountryCode,Date,School closing,Stay at home requirements,ConfirmedCases,ConfirmedDeaths,StringencyIndex,Continent_Name
0,Aruba,ABW,20200301,0,0,NaN,NaN,0.0,North America
1,Aruba,ABW,20200302,0,0,NaN,NaN,0.0,North America
2,Aruba,ABW,20200303,0,0,NaN,NaN,0.0,North America
3,Aruba,ABW,20200304,0,0,NaN,NaN,0.0,North America
4,Aruba,ABW,20200305,0,0,NaN,NaN,0.0,North America


After the merge there are 14499 records, prior to the merge there were 14013. Therefore, some duplication occurred during the merge, this is due to the transcontinental countries, as stated in the question no action was taken for these multiple entries.

In [14]:
OxCGRT_cont_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14499 entries, 0 to 14498
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CountryName                14499 non-null  object 
 1   CountryCode                14499 non-null  object 
 2   Date                       14499 non-null  int64  
 3   School closing             14499 non-null  int64  
 4   Stay at home requirements  14499 non-null  int64  
 5   ConfirmedCases             12348 non-null  float64
 6   ConfirmedDeaths            12348 non-null  float64
 7   StringencyIndex            14499 non-null  float64
 8   Continent_Name             14418 non-null  object 
dtypes: float64(3), int64(3), object(3)
memory usage: 1.1+ MB


Determine the country with missing continent name

In [15]:
print((OxCGRT_cont_df.loc[OxCGRT_cont_df.Continent_Name.isnull()].CountryName).unique())

['Kosovo']


Kosovo is a European country. Determine the syntax and spelling of Europe in the continent name column

In [16]:
OxCGRT_cont_df.Continent_Name.unique()

array(['North America', 'Asia', 'Africa', 'Europe', 'South America',
       'Oceania', nan], dtype=object)

Since only records with Kosovo have missing continent values, fill all NaN values in continent name column with 'Europe' 

In [17]:
OxCGRT_cont_df['Continent_Name'] = OxCGRT_cont_df.Continent_Name.fillna('Europe')

Check if the nulls have been replaced successfully 

In [18]:
OxCGRT_cont_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14499 entries, 0 to 14498
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CountryName                14499 non-null  object 
 1   CountryCode                14499 non-null  object 
 2   Date                       14499 non-null  int64  
 3   School closing             14499 non-null  int64  
 4   Stay at home requirements  14499 non-null  int64  
 5   ConfirmedCases             12348 non-null  float64
 6   ConfirmedDeaths            12348 non-null  float64
 7   StringencyIndex            14499 non-null  float64
 8   Continent_Name             14499 non-null  object 
dtypes: float64(3), int64(3), object(3)
memory usage: 1.1+ MB


In [ ]:
OxCGRT_cont_df.loc[OxCGRT_cont_df.CountryName == 'Kosovo']

,CountryName,CountryCode,Date,School closing,Stay at home requirements,ConfirmedCases,ConfirmedDeaths,StringencyIndex,Continent_Name
5113,Kosovo,RKS,20200301,0,0,0.0,0.0,0.00,Europe
5114,Kosovo,RKS,20200302,0,0,0.0,0.0,0.00,Europe
5115,Kosovo,RKS,20200303,0,0,0.0,0.0,0.00,Europe
5116,Kosovo,RKS,20200304,0,0,0.0,0.0,0.00,Europe
5117,Kosovo,RKS,20200305,0,0,0.0,0.0,0.00,Europe
...,...,...,...,...,...,...,...,...,...
12826,Kosovo,RKS,20200516,3,2,944.0,29.0,96.43,Europe
12827,Kosovo,RKS,20200517,3,2,955.0,29.0,96.43,Europe
12828,Kosovo,RKS,20200518,3,2,955.0,29.0,91.67,Europe
12829,Kosovo,RKS,20200519,3,2,989.0,29.0,91.67,Europe


### Handle missing values

Determine the number of missing values in confirmed cases column

In [ ]:
OxCGRT_cont_df.loc[OxCGRT_cont_df.ConfirmedCases.isnull()].shape

(2151, 9)

Determine the number of missing values in confirmed deaths column

In [ ]:
OxCGRT_cont_df.loc[OxCGRT_cont_df.ConfirmedDeaths.isnull()].shape

(2151, 9)

The confirmed cases and confirmed deaths appear to be missing for the same records. The data is missing across countries and dates.

In [ ]:
OxCGRT_cont_df.loc[OxCGRT_cont_df.ConfirmedDeaths.isnull()]

,CountryName,CountryCode,Date,School closing,Stay at home requirements,ConfirmedCases,ConfirmedDeaths,StringencyIndex,Continent_Name
0,Aruba,ABW,20200301,0,0,NaN,NaN,0.00,North America
1,Aruba,ABW,20200302,0,0,NaN,NaN,0.00,North America
2,Aruba,ABW,20200303,0,0,NaN,NaN,0.00,North America
3,Aruba,ABW,20200304,0,0,NaN,NaN,0.00,North America
4,Aruba,ABW,20200305,0,0,NaN,NaN,0.00,North America
...,...,...,...,...,...,...,...,...,...
11743,Rwanda,RWA,20200314,0,2,NaN,NaN,45.95,Africa
11840,Solomon Islands,SLB,20200520,2,2,NaN,NaN,61.90,Oceania
12537,Venezuela,VEN,20200313,0,2,NaN,NaN,53.57,South America
12538,Venezuela,VEN,20200314,0,2,NaN,NaN,53.57,South America


Create a dataframe displaying the count of records per country

In [ ]:
total_record_count_df = OxCGRT_cont_df.groupby(['CountryCode'])[['Date']].count().reset_index().rename(columns = {'Date':'Total_count'})
total_record_count_df

,CountryCode,Total_count
0,ABW,81
1,AFG,81
2,AGO,81
3,ALB,81
4,AND,81
...,...,...
168,VNM,81
169,YEM,81
170,ZAF,81
171,ZMB,81


Create a dataframe displaying the count of missing values per country

In [ ]:
total_missin_count_df = OxCGRT_cont_df.loc[OxCGRT_cont_df.ConfirmedCases.isnull()].groupby(['CountryCode'])[['Date']].count().reset_index().rename(columns = {'Date':'Missing_count'})
total_missin_count_df 

,CountryCode,Missing_count
0,ABW,22
1,AFG,10
2,AGO,21
3,ALB,8
4,AND,13
...,...,...
141,VNM,4
142,YEM,40
143,ZAF,7
144,ZMB,18


Merge the two dataframes and create a new column showing the percentage of missing values

In [ ]:
missing_values_df = total_record_count_df.merge(total_missin_count_df, left_on = ['CountryCode'], right_on = ['CountryCode'])

In [ ]:
missing_values_df['percent_missing'] = missing_values_df.Missing_count / missing_values_df.Total_count

In [ ]:
missing_values_df.sort_values(['percent_missing'], ascending=False)

,CountryCode,Total_count,Missing_count,percent_missing
113,SLB,81,81,1.000000
129,TKM,81,81,1.000000
75,LSO,81,75,0.925926
128,TJK,81,61,0.753086
142,YEM,81,40,0.493827
...,...,...,...,...
104,PRT,81,2,0.024691
59,IND,81,1,0.012346
89,MYS,81,1,0.012346
116,SMR,81,1,0.012346


Calculate the average percentage of missing value across the data.

In [ ]:
missing_values_df['percent_missing'].mean() 

0.17782851344495168

In [ ]:
OxCGRT_cont_df.loc[OxCGRT_cont_df.CountryCode == 'ABW']

,CountryName,CountryCode,Date,School closing,Stay at home requirements,ConfirmedCases,ConfirmedDeaths,StringencyIndex,Continent_Name
0,Aruba,ABW,20200301,0,0,NaN,NaN,0.00,North America
1,Aruba,ABW,20200302,0,0,NaN,NaN,0.00,North America
2,Aruba,ABW,20200303,0,0,NaN,NaN,0.00,North America
3,Aruba,ABW,20200304,0,0,NaN,NaN,0.00,North America
4,Aruba,ABW,20200305,0,0,NaN,NaN,0.00,North America
...,...,...,...,...,...,...,...,...,...
7899,Aruba,ABW,20200519,0,2,101.0,3.0,50.00,North America
7900,Aruba,ABW,20200520,0,2,101.0,3.0,50.00,North America
12831,Aruba,ABW,20200410,3,3,82.0,0.0,82.14,North America
12832,Aruba,ABW,20200412,3,3,92.0,0.0,82.14,North America


Replace NaN in confirmed deaths columns with zero

In [ ]:
OxCGRT_cont_df['ConfirmedDeaths'] = OxCGRT_cont_df.ConfirmedDeaths.fillna(0)

Replace NaN in confirmed cases columns with zero

In [ ]:
OxCGRT_cont_df['ConfirmedCases'] = OxCGRT_cont_df.ConfirmedCases.fillna(0)

In [ ]:
OxCGRT_cont_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14499 entries, 0 to 14498
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CountryName                14499 non-null  object 
 1   CountryCode                14499 non-null  object 
 2   Date                       14499 non-null  int64  
 3   School closing             14499 non-null  int64  
 4   Stay at home requirements  14499 non-null  int64  
 5   ConfirmedCases             14499 non-null  float64
 6   ConfirmedDeaths            14499 non-null  float64
 7   StringencyIndex            14499 non-null  float64
 8   Continent_Name             14499 non-null  object 
dtypes: float64(3), int64(3), object(3)
memory usage: 1.1+ MB


The animation_frame parameter of the px.scatter_geo function does not accept columns in date time formats. Therefore, the Date column is kept as an integer and the dataframe is ordered by date to ensure the geo map animation plays in chronological order.

In [ ]:
OxCGRT_cont_df = OxCGRT_cont_df.sort_values(['Date'], ascending=True)

Create a list of dictionaries for the continents dropdown (scope).

In [ ]:
continent_list = [{'label':'World', 'value': 'World'},
                  {'label':'Asia', 'value': 'Asia'},
                  {'label':'Africa', 'value': 'Africa'},
                  {'label':'Europe', 'value': 'Europe'},
                  {'label':'North America', 'value': 'North America'},
                  {'label':'South America', 'value': 'South America'}
                 ]

continent_list

[{'label': 'World', 'value': 'World'},
 {'label': 'Asia', 'value': 'Asia'},
 {'label': 'Africa', 'value': 'Africa'},
 {'label': 'Europe', 'value': 'Europe'},
 {'label': 'North America', 'value': 'North America'},
 {'label': 'South America', 'value': 'South America'}]

Create a list of dictionaries for the data input dropdown. 

In [ ]:
data_input_list = [{'label':'Confirmed Cases', 'value': 'ConfirmedCases'},
                  {'label':'Confirmed Deaths', 'value': 'ConfirmedDeaths'},
                  {'label':'Stringency Index', 'value': 'StringencyIndex'}
                 ]

data_input_list

[{'label': 'Confirmed Cases', 'value': 'ConfirmedCases'},
 {'label': 'Confirmed Deaths', 'value': 'ConfirmedDeaths'},
 {'label': 'Stringency Index', 'value': 'StringencyIndex'}]

Create a list of dictionaries for the policies dropdown.

In [ ]:
policies_list = [{'label':'Not selected (default)', 'value': 'Default'},
                  {'label':'School closing', 'value': 'School closing'},
                  {'label':'Staying at home', 'value': 'Stay at home requirements'}
                 ]

policies_list

[{'label': 'Not selected (default)', 'value': 'Default'},
 {'label': 'School closing', 'value': 'School closing'},
 {'label': 'Staying at home', 'value': 'Stay at home requirements'}]

Create a new date column in datetime format for later use

In [ ]:
OxCGRT_cont_df['Date_2'] = OxCGRT_cont_df.Date.map(lambda x: datetime.strptime(str(x),'%Y%m%d'))

In [ ]:
OxCGRT_cont_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14499 entries, 0 to 11454
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   CountryName                14499 non-null  object        
 1   CountryCode                14499 non-null  object        
 2   Date                       14499 non-null  int64         
 3   School closing             14499 non-null  int64         
 4   Stay at home requirements  14499 non-null  int64         
 5   ConfirmedCases             14499 non-null  float64       
 6   ConfirmedDeaths            14499 non-null  float64       
 7   StringencyIndex            14499 non-null  float64       
 8   Continent_Name             14499 non-null  object        
 9   Date_2                     14499 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(3)
memory usage: 1.2+ MB


Create a dataframe for each of the following measures in the original dataframe:
- Confirmed Cases
- Confirmed Death
- Stringency Index
- School closing
- Stay at home requirements

To create the graphs that will be built using these dataframes a new column called KPI will be created to indicate the name of the measurement.

In [ ]:
cases_df = OxCGRT_cont_df[['CountryName','CountryCode','Continent_Name','Date','Date_2','ConfirmedCases']]
cases_df['ContinentNamelower'] = cases_df.Continent_Name.str.lower()
cases_df['KPI'] = 'Confirmed Cases'
cases_df.columns = ['CountryName','CountryCode','Continent_Name','Date','Date_2','measure','ContinentNamelower','KPI']

deaths_df = OxCGRT_cont_df[['CountryName','CountryCode','Continent_Name','Date','Date_2','ConfirmedDeaths']]
deaths_df['ContinentNamelower'] = deaths_df.Continent_Name.str.lower()
deaths_df['KPI'] = 'Confirmed Deaths'
deaths_df.columns = ['CountryName','CountryCode','Continent_Name','Date','Date_2','measure','ContinentNamelower','KPI']

stringency_df = OxCGRT_cont_df[['CountryName','CountryCode','Continent_Name','Date','Date_2','StringencyIndex']]
stringency_df['ContinentNamelower'] = stringency_df.Continent_Name.str.lower()
stringency_df['KPI'] = 'Stringency Index'
stringency_df.columns = ['CountryName','CountryCode','Continent_Name','Date','Date_2','measure','ContinentNamelower','KPI']

school_df = OxCGRT_cont_df[['CountryName','CountryCode','Continent_Name','Date','Date_2','School closing']]
school_df['ContinentNamelower'] = school_df.Continent_Name.str.lower()
school_df['KPI'] = 'School closing'
school_df.columns = ['CountryName','CountryCode','Continent_Name','Date','Date_2','measure','ContinentNamelower','KPI']

home_df = OxCGRT_cont_df[['CountryName','CountryCode','Continent_Name','Date','Date_2','Stay at home requirements']]
home_df['ContinentNamelower'] = home_df.Continent_Name.str.lower()
home_df['KPI'] = 'Stay at home requirements'
home_df.columns = ['CountryName','CountryCode','Continent_Name','Date','Date_2','measure','ContinentNamelower','KPI']


C:\Users\nirun\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\nirun\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\nirun\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy 

Create a function to produce the bubble world map. This function will be used to generate the bubble graph world map.

In [ ]:
def scatter_fig_def(df):

    world_graph_fig = px.scatter_geo(df,
                                     locations="CountryCode",
                                     color="Continent_Name",
                                     hover_name="CountryName", 
                                     size= "measure", 
                                     size_max=25,
                                     animation_frame="Date",
                                     projection="natural earth",
                                     title=df.KPI.unique().item() + " in world",
                                     opacity=0.6) 
    
    world_graph_fig.update_layout(height=600)
    
    
    
    return world_graph_fig


Create a function to produce the bubble world map. This function will be used to generate the bubble graph for each continent.

In [ ]:
def scatter_scope_fig_def(df):

    world_graph_fig = px.scatter_geo(df,
                                     locations="CountryCode",
                                     color="Continent_Name",
                                     hover_name="CountryName", 
                                     size= "measure", 
                                     size_max=25,
                                     scope=df.ContinentNamelower.unique().item(), 
                                     animation_frame="Date",
                                     projection="natural earth",
                                     title=df.KPI.unique().item() + " in " + df.Continent_Name.unique().item(),
                                     opacity=0.6) 
    
    world_graph_fig.update_layout(height=600)
    
    
    
    return world_graph_fig


Create a function to produce the choropleth graph. This function will be used to generate the choropleth graph world map.

In [ ]:
def chloreopath_fig_def(df):
    
    world_graph_fig = px.choropleth(df,
                                    locations="CountryCode",
                                    color="measure",
                                    hover_name="CountryName",
                                    animation_frame="Date",
                                    title=df.KPI.unique().item() + " in world",
                                    color_continuous_scale=px.colors.sequential.Plasma)

    world_graph_fig.update_layout(height=600)
    
    
    return world_graph_fig

Create a function to produce the choropleth graph. This function will be used to generate the choropleth graph for each continent. 

In [ ]:
def chloreopath_scope_fig_def(df):
    
    world_graph_fig = px.choropleth(df,
                                    locations="CountryCode",
                                    color="measure",
                                    hover_name="CountryName",
                                    animation_frame="Date",
                                    scope=df.ContinentNamelower.unique().item(), 
                                    title=df.KPI.unique().item() + " in " + df.Continent_Name.unique().item(),
                                    color_continuous_scale=px.colors.sequential.Plasma)
    
    world_graph_fig.update_layout(height=600)
   
    return world_graph_fig

Determine top 5 countries with highest confirmed cases

In [ ]:
top_five = OxCGRT_cont_df.loc[OxCGRT_cont_df.Date==20200520].nlargest(10, 'ConfirmedCases').CountryCode.unique()[:5]
top_five

array(['USA', 'RUS', 'BRA', 'GBR', 'ESP'], dtype=object)

Create a list for y axis labels

In [ ]:
y_axis_integer = [0 , 1, 2, 3]

school_description = ['no measures' , 'recommend closing' ,'require some closing' ,'require all closing']

home_description = ['no measures', 'recommend not leaving house', 'require not leaving house (incl. exceptions)',  'require not leaving house (min exceptions)']

Create a function to produce the line graph.

In [ ]:
def line_graph_def(df,policy):
    
    
    
    if policy ==  'School closing':
        
        line_graph = px.line(
            df,
            x="Date_2", 
            y="measure",
            color="CountryName",
            #log_y="measure",
            title=df.KPI.unique().item()    )

        line_graph.update_layout(

            title=df.KPI.unique().item() + " in the top five countries",
            yaxis_title=df.KPI.unique().item(),
            height=600

        )

        line_graph.update_yaxes(
            ticktext=school_description, 
            tickvals=y_axis_integer)  
        return line_graph
    
    elif policy == 'Stay at home requirements':
        
        line_graph = px.line(
            df,
            x="Date_2", 
            y="measure",
            color="CountryName",
            #log_y="measure",
            title=df.KPI.unique().item()    )

        line_graph.update_layout(

            title=df.KPI.unique().item() + " in the top five countries",
            yaxis_title=df.KPI.unique().item(),
            height=600

        )

        line_graph.update_yaxes(
            ticktext=home_description, 
            tickvals=y_axis_integer)  
        return line_graph

    else:
        
        line_graph = px.line(
            df,
            x="Date_2", 
            y="measure",
            color="CountryName",
            log_y="measure",
            title=df.KPI.unique().item()    )

        line_graph.update_layout(

            title=df.KPI.unique().item() + " in the top five countries",
            yaxis_title=df.KPI.unique().item(),
            height=600

        )

        return line_graph

Reuse dash app from question 4 but with additional line graph

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets) 

app.layout = html.Div([
    
    
    html.Div([
        html.H1("Covid-19 World Dashboard"),
        html.P("An interactive dashboard demonstrating key covid-19 metrics. Report includes data from 01/03/2020 to 20/05/2020")
        ], 
        style = {'padding' : '0px' , 'backgroundColor' : '#9bba59'}),
    
    
    html.Div([
        html.Div([
            html.Label("Scope"),
            dcc.Dropdown(id='Continent-dropdown',options=continent_list,value="World"),
            html.Label('_'),    
            html.Label('Input Data'),         
            dcc.RadioItems(id="Data-Input-dropdown",options=data_input_list,value="ConfirmedCases"),
            html.Label('_'),    
            html.Label('Policy'),         
            dcc.RadioItems(id="Policies-dropdown",options=policies_list,value="Default")
            ], className="two columns" ),
        html.Div([
            dcc.Graph(id='Line_Graph')
            ], className="ten columns"),
        html.Div([
            dcc.Graph(id='World_Graph')
            ], className="twelve columns"),
        ], className="row")
    ])



@app.callback(Output('World_Graph', 'figure'),
              [Input('Continent-dropdown', 'value'),
               Input('Data-Input-dropdown', 'value'),
               Input('Policies-dropdown', 'value')
              ])


def show_graph(continent, data, policy):
    
    if continent == 'World':
        if policy == 'Default':
            if data == 'ConfirmedCases':
                return scatter_fig_def(cases_df)
            elif data == 'ConfirmedDeaths':
                return scatter_fig_def(deaths_df)
            elif data == 'StringencyIndex':
                return scatter_fig_def(stringency_df)
        elif policy != 'Default':
            if policy == 'School closing':
                return chloreopath_fig_def(school_df)
            elif policy == 'Stay at home requirements':
                return chloreopath_fig_def(home_df)
    elif continent != 'World':
        if policy == 'Default':
            if data == 'ConfirmedCases':
                return  scatter_scope_fig_def(cases_df[cases_df.Continent_Name == continent])
            elif data == 'ConfirmedDeaths':
                return  scatter_scope_fig_def(deaths_df[deaths_df.Continent_Name == continent])
            elif data == 'StringencyIndex':
                return  scatter_scope_fig_def(stringency_df[stringency_df.Continent_Name == continent])
        elif policy == 'School closing':
            return  chloreopath_scope_fig_def(school_df[school_df.Continent_Name == continent])
        elif policy == 'Stay at home requirements':
            return  chloreopath_scope_fig_def(home_df[home_df.Continent_Name == continent]) 


@app.callback(Output('Line_Graph', 'figure'),
              [Input('Data-Input-dropdown', 'value'),
               Input('Policies-dropdown', 'value')
              ])

def show_line_graph(data, policy):
    
    if policy == 'Default':
        if data == 'ConfirmedCases':
            return  line_graph_def(cases_df[cases_df.CountryCode.isin(top_five)],policy)
        elif data == 'ConfirmedDeaths':
            return  line_graph_def(deaths_df[deaths_df.CountryCode.isin(top_five)],policy)
        elif data == 'StringencyIndex':
            return  line_graph_def(stringency_df[stringency_df.CountryCode.isin(top_five)],policy)
    elif policy == 'School closing':
        return  line_graph_def(school_df[school_df.CountryCode.isin(top_five)],policy)
    elif policy == 'Stay at home requirements':
        return  line_graph_def(home_df[home_df.CountryCode.isin(top_five)],policy)
    
if __name__ == '__main__':
    app.run_server(debug=False)
    
    

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jul/2020 15:38:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 15:38:57] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 15:38:57] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 15:38:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 15:39:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 15:39:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 15:39:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 15:39:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 15:39:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 15:39:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 15:39:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 15:39:14] "POST /_dash-update-component